In [402]:
import yfinance as yf
import pandas as pd

ticker = yf.Ticker('TSLA')  #PRX.AS ASML.AS
raw_df = ticker.history(start='2010-01-01', end="2022-01-01", interval="1d")
ticker.info["longName"]

'Tesla, Inc.'

In [439]:
def to_signal(returns):
    if returns >= 0.03:
        return 1
    if returns <= 0.0:
        return -1
    return 0

In [448]:
raw_df['returnsM'] = raw_df[['Close']].pct_change(periods=30)
df = raw_df[['Open', 'High', 'Low', 'Close', 'Volume', 'returnsM']]
df['returnsM_shift'] = df['returnsM'].shift(periods=-30)
df['signal'] = df.apply(lambda r: to_signal(r['returnsM_shift']), axis=1)
df.drop(columns=['returnsM', 'returnsM_shift'], inplace=True)
df

/tmp/ipykernel_140819/82568990.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['returnsM_shift'] = df['returnsM'].shift(periods=-30)


,Open,High,Low,Close,Volume,signal
Date,,,,,,
2010-06-29,3.800000,5.000000,3.508000,4.778000,93831500,-1
2010-06-30,5.158000,6.084000,4.660000,4.766000,85935500,-1
2010-07-01,5.000000,5.184000,4.054000,4.392000,41094000,-1
2010-07-02,4.600000,4.620000,3.742000,3.840000,25699000,-1
2010-07-06,4.000000,4.000000,3.166000,3.222000,34334500,1
...,...,...,...,...,...,...
2021-12-27,1073.670044,1117.000000,1070.719971,1093.939941,23715300,0
2021-12-28,1109.489990,1119.000000,1078.420044,1088.469971,20108000,0
2021-12-29,1098.640015,1104.000000,1064.140015,1086.189941,18718000,0


In [449]:
from ta import add_all_ta_features
from ta.utils import dropna

# Clean NaN values
df = dropna(df)

# Add all ta features
df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume")

/home/mrrob0t/.local/lib/python3.10/site-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/mrrob0t/.local/lib/python3.10/site-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/home/mrrob0t/.local/lib/python3.10/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/home/mrrob0t/.local/lib/python3.10/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


In [450]:
df = df.fillna(method='backfill')
df = pd.concat([df[['Open', 'High', 'Low', 'Close', 'Volume', 'signal']], df.loc[:,df.columns.str.startswith('momentum')|df.columns.str.startswith('trend')]], join = 'outer', axis = 1)
#df = pd.concat([df[['Open', 'High', 'Low', 'Close', 'Volume', 'signal']], df.loc[:,df.columns.str.startswith('momentum')]], join = 'outer', axis = 1)
df

,Open,High,Low,Close,Volume,signal,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,...,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama
Date,,,,,,,,,,,,,,,,,,,,,
2010-06-29,3.800000,5.000000,3.508000,4.778000,93831500,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-06-30,5.158000,6.084000,4.660000,4.766000,85935500,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-07-01,5.000000,5.184000,4.054000,4.392000,41094000,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-07-02,4.600000,4.620000,3.742000,3.840000,25699000,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-07-06,4.000000,4.000000,3.166000,3.222000,34334500,1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-09,1173.599976,1174.500000,1011.520020,1023.500000,59105800,-1.0,97.158217,94.166125,2.992092,1115.824992,...,-56.699914,266.521652,14.485459,9.717097,9.832123,-0.115027,17.443660,15.314115,2.129545,1188.374865
2021-11-10,1010.409973,1078.099976,987.309998,1067.949951,42802700,-1.0,87.641774,92.861255,-5.219481,1129.014155,...,-45.243445,227.840766,17.398421,8.721339,9.609966,-0.888627,17.452269,15.741746,1.710523,1187.179661
2021-11-12,1047.500000,1054.500000,1019.200012,1033.420044,25573100,1.0,76.432582,89.575520,-13.142938,1130.263326,...,-59.589245,180.815003,1.471878,7.589950,9.205963,-1.616013,13.467855,15.286968,-1.819112,1185.115210


## Split data

In [451]:
train_df = df.loc[:'2016-01-01']
val_df = df.loc['2016-06-01':'2019-01-01']
test_df = df.loc['2019-01-01':'2021-11-01']
train_df


,Open,High,Low,Close,Volume,signal,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,...,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama
Date,,,,,,,,,,,,,,,,,,,,,
2010-06-29,3.800000,5.000000,3.508000,4.778000,93831500,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-06-30,5.158000,6.084000,4.660000,4.766000,85935500,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-07-01,5.000000,5.184000,4.054000,4.392000,41094000,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-07-02,4.600000,4.620000,3.742000,3.840000,25699000,-1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
2010-07-06,4.000000,4.000000,3.166000,3.222000,34334500,1.0,-0.154869,-0.124399,0.034803,3.856167,...,-65.544046,-0.113818,-8.287985,-3.842930,-3.135769,0.857460,-67.067426,-69.017320,0.943612,3.628000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-24,46.112000,46.375999,45.655998,46.113998,3540000,-1.0,0.334260,0.254608,0.079652,45.490333,...,-31.411097,1.626870,1.698122,0.735423,0.561279,0.174144,-17.535770,-10.591427,-6.944343,45.009883
2015-12-28,46.298000,46.396000,45.108002,45.790001,9506500,-1.0,0.309843,0.265655,0.044188,45.564167,...,-38.488370,1.349941,1.973102,0.681325,0.585288,0.096037,-17.886943,-12.050530,-5.836413,45.085976
2015-12-29,46.012001,47.543999,45.910000,47.438000,12031500,-1.0,0.418646,0.296253,0.122393,45.732833,...,-2.490136,1.226994,4.456771,0.917643,0.651759,0.265884,-16.311321,-12.902688,-3.408633,45.423242


## Normalize data

In [452]:
from sklearn.preprocessing import StandardScaler

def std_x_y(df):
    #scaler = StandardScaler()
    X = df.drop(columns=['signal']).to_numpy()
    #scaler.fit(X)
    #X = scaler.fit_transform(X)
    Y = df['signal'].to_numpy()
    return X, Y

x_train, y_train = std_x_y(train_df)
x_val, y_val = std_x_y(val_df)
x_test, y_test = std_x_y(test_df)

## Train

In [453]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

#n_folds = 5
#folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=666)
#
#best_model = -1
#
#for n_fold, (train_idx, cv_idx) in enumerate(folds.split(x_train, y_train)):
#    best_score = -1
#    # Train data
#    train_feats = x_train[train_idx]
#    train_lbls = y_train[train_idx]
#
#    # Cross-validation data
#    cv_feats = x_train[cv_idx]
#    cv_lbls = y_train[cv_idx]
#
#    clf = CatBoostClassifier(
#        iterations=3000, 
#        learning_rate=0.04,
#        custom_loss=['AUC', 'Accuracy']
#    )
#
#    clf.fit(train_feats, train_lbls, eval_set=[(cv_feats, cv_lbls)], verbose=1000)
#    score = clf.score(cv_feats, cv_lbls)
#
#    if score > best_score:
#        best_score = score
#        best_model = n_fold
#
#    clf.save_model('models/model_{}'.format(n_fold))
#
#clf = CatBoostClassifier()
#clf.load_model('models/model_{}'.format(best_model))

clf = CatBoostClassifier(
    iterations=3000, 
    learning_rate=0.08,
    #custom_loss=['AUC', 'Accuracy']
)

clf.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=1000)


0:	learn: 0.6581455	test: 0.6857203	best: 0.6857203 (0)	total: 6.03ms	remaining: 18.1s
1000:	learn: 0.0033347	test: 2.8357660	best: 0.6841528 (2)	total: 5.2s	remaining: 10.4s
2000:	learn: 0.0028673	test: 2.9184460	best: 0.6841528 (2)	total: 10.8s	remaining: 5.38s
2999:	learn: 0.0027174	test: 2.9286938	best: 0.6841528 (2)	total: 18.4s	remaining: 0us

bestTest = 0.6841528316
bestIteration = 2

Shrink model to first 3 iterations.


In [454]:
test_df['signal_pred'] = clf.predict(x_test)

/tmp/ipykernel_140819/3157580224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['signal_pred'] = clf.predict(x_test)


## Backtest

In [455]:
import math

ini_capital = 10000
capital = ini_capital
shares = 0
hold_duration = 0
max_hold = 30

test_arr = test_df[['Close', 'signal_pred']].reset_index().to_numpy()

for idx, data in enumerate(test_arr):
    date, price, sig = data[0].date(), data[1], data[2]

    if (capital + shares * price) < price:
        print('chapter 11')
        break

    if shares == 0 and sig == 1:
        quant = math.floor(capital/price)
        shares = quant
        capital = capital - shares * price

    if shares > 0:
        hold_duration += 1
        if sig == -1 or hold_duration >= max_hold or idx == len(test_arr)-1:
            capital = capital + shares * price
            shares = 0
    
print((capital-ini_capital)/ini_capital)

0.0645334129333496


## Benchmark

In [432]:
capital = ini_capital
shares = 0
hold_duration = 0
max_hold = 30

test_arr = test_df[['Close', 'signal_pred']].reset_index().to_numpy()


capital = capital * (1 + (test_arr[len(test_arr)-1][1]-test_arr[0][1])/test_arr[0][1])
print((capital-ini_capital)/ini_capital)



18.48584418476681
